In [1]:
import requests
import xml.etree.ElementTree as ET
import re
import time
import zipfile
import os
from google.colab import files

# ==========================================
# 設定：取得したい法令名をリストで指定
# ==========================================
TARGET_LAWS = [
    # --- 1. 建築・都市計画の基本 ---
    "建築基準法",
    "建築基準法施行令",
    "建築基準法施行規則",

    "都市計画法",
    "都市計画法施行令",
    "都市計画法施行規則",

    "都市緑地法",
    "都市緑地法施行令",
    "都市緑地法施行規則",

    "流通業務市街地の整備に関する法律",
    "流通業務市街地の整備に関する法律施行令",
    "流通業務市街地の整備に関する法律施行規則",

    "宅地造成及び特定盛土等規制法", # 盛土規制法
    "宅地造成及び特定盛土等規制法施行令",
    "宅地造成及び特定盛土等規制法施行規則",

    "景観法", # ★追加：外観・高さ制限などで建築基準法と密接に関係
    "景観法施行令",
    "景観法施行規則",

    # --- 2. 住宅・性能評価（★建築基準法をベースに基準設定・準用） ---
    "住宅の品質確保の促進等に関する法律", # 品確法
    "住宅の品質確保の促進等に関する法律施行令",
    "住宅の品質確保の促進等に関する法律施行規則",

    "長期優良住宅の普及の促進に関する法律", # 長期優良住宅法
    "長期優良住宅の普及の促進に関する法律施行令",
    "長期優良住宅の普及の促進に関する法律施行規則",

    "住宅宿泊事業法", # 民泊新法（建築基準法の安全規定を参照）
    "住宅宿泊事業法施行令",
    "住宅宿泊事業法施行規則",

    # --- 3. 省エネ・リサイクル ---
    "建築物のエネルギー消費性能の向上等に関する法律", # 建築物省エネ法
    "建築物のエネルギー消費性能の向上等に関する法律施行令",
    "建築物のエネルギー消費性能の向上等に関する法律施行規則",

    "建設工事に係る資材の再資源化等に関する法律", # 建設リサイクル法
    "建設工事に係る資材の再資源化等に関する法律施行令",
    "建設工事に係る資材の再資源化等に関する法律施行規則",

    # --- 4. 消防・危険物・産業保安 ---
    "消防法",
    "消防法施行令",
    "消防法施行規則",

    "液化石油ガスの保安の確保及び取引の適正化に関する法律",
    "液化石油ガスの保安の確保及び取引の適正化に関する法律施行令",
    "液化石油ガスの保安の確保及び取引の適正化に関する法律施行規則",

    "高圧ガス保安法",
    "高圧ガス保安法施行令",

    "ガス事業法",
    "ガス事業法施行令",
    "ガス事業法施行規則",

    "労働安全衛生法", # ★追加：工場建屋や機械設備（昇降機等）で重複・関連
    "労働安全衛生法施行令",
    "労働安全衛生規則", # ※施行規則ではなく「規則」が正式名称

    # --- 5. 水・衛生・環境 ---
    "水道法",
    "水道法施行令",
    "水道法施行規則",

    "下水道法",
    "下水道法施行令",
    "下水道法施行規則",

    "浄化槽法",
    "浄化槽法施行令",
    "環境省関係浄化槽法施行規則",

    "建築物における衛生的環境の確保に関する法律", # ★追加：いわゆるビル管法（建築後の維持管理だが、設備設計に関係）
    "建築物における衛生的環境の確保に関する法律施行令",
    "建築物における衛生的環境の確保に関する法律施行規則",

    "特定都市河川浸水被害対策法",
    "特定都市河川浸水被害対策法施行令",
    "特定都市河川浸水被害対策法施行規則",

    # --- 6. 交通・駐車場・港湾・空港 ---
    "駐車場法",
    "駐車場法施行令",
    "駐車場法施行規則",

    "自転車の安全利用の促進及び自転車等の駐車対策の総合的推進に関する法律",
    "自転車の安全利用の促進及び自転車等の駐車対策の総合的推進に関する法律施行令",
    "自転車の安全利用の促進及び自転車等の駐車対策の総合的推進に関する法律施行規則",

    "港湾法",
    "港湾法施行令",
    "港湾法施行規則",

    "特定空港周辺航空機騒音対策特別措置法",
    "特定空港周辺航空機騒音対策特別措置法施行令",
    "特定空港周辺航空機騒音対策特別措置法施行規則",

    # --- 7. 福祉・その他 ---
    "高齢者、障害者等の移動等の円滑化の促進に関する法律", # バリアフリー法
    "高齢者、障害者等の移動等の円滑化の促進に関する法律施行令",
    "高齢者、障害者等の移動等の円滑化の促進に関する法律施行規則",

    # --- 8. 営業許可関連（建築確認済証が必要となる主な業法） ---
    "旅館業法",
    "旅館業法施行令",
    "旅館業法施行規則",

    "興行場法",
    "興行場法施行規則",
]
# ==========================================

def sanitize_filename(name):
    """ファイル名に使えない文字を置換"""
    return re.sub(r'[\\|/|:|?|"|<|>|\|]', '_', name)

def get_law_id_map():
    """
    全法令リストを取得し、{法令名: LawId} の辞書を作成して返す
    """
    print("全法令リストを取得しています...")
    list_url = "https://elaws.e-gov.go.jp/api/1/lawlists/1"

    try:
        response = requests.get(list_url, timeout=30) # リストが大きいのでタイムアウト長め
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error: 法令リストの取得に失敗しました。\n{e}")
        return None

    try:
        root = ET.fromstring(response.content)
    except ET.ParseError:
        print("Error: XMLの解析に失敗しました。")
        return None

    # 辞書 {法令名: LawId} を作成
    law_map = {}
    for law in root.findall(".//LawNameListInfo"):
        name_tag = law.find("LawName")
        id_tag = law.find("LawId")
        if name_tag is not None and id_tag is not None:
            law_map[name_tag.text] = id_tag.text

    print(f"リスト取得完了（{len(law_map)}件の法令が見つかりました）")
    return law_map

def fetch_law_content(law_id, law_name):
    """指定されたIDの条文データを取得"""
    print(f"[{law_name}] の条文データをダウンロード中... (ID: {law_id})")
    detail_url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{law_id}"

    try:
        response = requests.get(detail_url, timeout=20)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Error: {law_name} の取得に失敗しました。\n{e}")
        return None

def parse_to_markdown(xml_content, law_name):
    """XMLをMarkdownに変換"""
    try:
        root = ET.fromstring(xml_content)
    except ET.ParseError:
        print(f"Error: {law_name} のXML解析に失敗しました。")
        return None

    markdown_text = f"# {law_name}\n\n"
    articles = root.findall(".//Article")

    for article in articles:
        # 条数と見出し
        caption = article.find("ArticleCaption")
        title = article.find("ArticleTitle")

        caption_txt = caption.text if caption is not None else ""
        title_txt = title.text if title is not None else ""

        markdown_text += f"## {title_txt} {caption_txt}\n"

        # 項
        for p in article.findall(".//Paragraph"):
            p_num = p.find("ParagraphNum")
            p_sent = p.find(".//Sentence")

            num_txt = p_num.text if p_num is not None else ""
            sent_txt = p_sent.text if p_sent is not None and p_sent.text else ""

            markdown_text += f"- **{num_txt}**: {sent_txt}\n"

            # 号
            for item in p.findall("Item"):
                i_title = item.find("ItemTitle")
                i_sent = item.find(".//Sentence")

                it_txt = i_title.text if i_title is not None else ""
                is_txt = i_sent.text if i_sent is not None else ""

                markdown_text += f"    - **{it_txt}**: {is_txt}\n"

                # 号の下の階層（イロハなど）
                for sub1 in item.findall("Subitem1"):
                    s1_title = sub1.find("Subitem1Title")
                    s1_sent = sub1.find(".//Sentence")

                    s1_t_txt = s1_title.text if s1_title is not None else ""
                    s1_s_txt = s1_sent.text if s1_sent is not None else ""

                    markdown_text += f"        - **{s1_t_txt}**: {s1_s_txt}\n"

        markdown_text += "\n"
    return markdown_text

# --- メイン実行処理 ---
if __name__ == "__main__":
    # 1. 法令IDマップを作成（通信1回）
    law_map = get_law_id_map()

    if law_map:
        generated_files = [] # 生成されたファイル名のリスト

        # 2. 指定された法令を順次処理
        for target_name in TARGET_LAWS:
            if target_name in law_map:
                law_id = law_map[target_name]

                # サーバーへの負荷軽減のため1秒待機
                time.sleep(1)

                # データ取得
                xml_data = fetch_law_content(law_id, target_name)

                if xml_data:
                    # Markdown変換
                    md_text = parse_to_markdown(xml_data, target_name)

                    if md_text:
                        # ファイル保存
                        safe_name = sanitize_filename(target_name)
                        filename = f"{safe_name}.md"
                        with open(filename, "w", encoding="utf-8") as f:
                            f.write(md_text)

                        generated_files.append(filename)
                        print(f"  -> 保存完了: {filename}")
            else:
                print(f"Warning: '{target_name}' は法令リストに見つかりませんでした。（名称が正確か確認してください）")

        # 3. ZIPにまとめてダウンロード
        if generated_files:
            zip_filename = "laws_markdown.zip"
            print(f"\n{len(generated_files)}個のファイルをZIPに圧縮しています...")

            with zipfile.ZipFile(zip_filename, 'w') as zipf:
                for file in generated_files:
                    zipf.write(file)

            print("ダウンロードを開始します...")
            try:
                files.download(zip_filename)
            except Exception as e:
                 print(f"ダウンロードの起動に失敗しました: {e}")
        else:
            print("ダウンロード対象のファイルが生成されませんでした。")

全法令リストを取得しています...
リスト取得完了（8918件の法令が見つかりました）
[建築基準法] の条文データをダウンロード中... (ID: 325AC0000000201)
  -> 保存完了: 建築基準法.md
[建築基準法施行令] の条文データをダウンロード中... (ID: 325CO0000000338)
  -> 保存完了: 建築基準法施行令.md
[建築基準法施行規則] の条文データをダウンロード中... (ID: 325M50004000040)
  -> 保存完了: 建築基準法施行規則.md
[都市計画法] の条文データをダウンロード中... (ID: 343AC0000000100)
  -> 保存完了: 都市計画法.md
[都市計画法施行令] の条文データをダウンロード中... (ID: 344CO0000000158)
  -> 保存完了: 都市計画法施行令.md
[都市計画法施行規則] の条文データをダウンロード中... (ID: 344M50004000049)
  -> 保存完了: 都市計画法施行規則.md
[都市緑地法] の条文データをダウンロード中... (ID: 348AC0000000072)
  -> 保存完了: 都市緑地法.md
[都市緑地法施行令] の条文データをダウンロード中... (ID: 349CO0000000003)
  -> 保存完了: 都市緑地法施行令.md
[都市緑地法施行規則] の条文データをダウンロード中... (ID: 349M50004000001)
  -> 保存完了: 都市緑地法施行規則.md
[流通業務市街地の整備に関する法律] の条文データをダウンロード中... (ID: 341AC0000000110)
  -> 保存完了: 流通業務市街地の整備に関する法律.md
[流通業務市街地の整備に関する法律施行令] の条文データをダウンロード中... (ID: 342CO0000000003)
  -> 保存完了: 流通業務市街地の整備に関する法律施行令.md
[流通業務市街地の整備に関する法律施行規則] の条文データをダウンロード中... (ID: 342M50004000003)
  -> 保存完了: 流通業務市街地の整備に関する法律施行規則.md
[宅地造成及び特定盛土等規制法] の

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>